In [1]:
! git clone https://github.com/zllrunning/face-parsing.PyTorch

Cloning into 'face-parsing.PyTorch'...
remote: Enumerating objects: 92, done.
remote: Total 92 (delta 0), reused 0 (delta 0), pack-reused 92
Unpacking objects: 100% (92/92), done.


In [2]:
! mkdir -p face-parsing.PyTorch/res/cp 

In [3]:
!wget --output-document=79999_iter.pth 'https://drive.google.com/u/0/uc?id=154JgKpzCPW82qINcVieuPH3fZ2e0P812&export=download'

--2020-07-13 12:24:48--  https://drive.google.com/u/0/uc?id=154JgKpzCPW82qINcVieuPH3fZ2e0P812&export=download
Resolving drive.google.com (drive.google.com)... 74.125.129.102, 74.125.129.139, 74.125.129.101, ...
Connecting to drive.google.com (drive.google.com)|74.125.129.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-1s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bjt2l1p0l7nhro1dpasn50mgm81j9hle/1594643025000/11347955420423960128/*/154JgKpzCPW82qINcVieuPH3fZ2e0P812?e=download [following]
--2020-07-13 12:24:50--  https://doc-10-1s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bjt2l1p0l7nhro1dpasn50mgm81j9hle/1594643025000/11347955420423960128/*/154JgKpzCPW82qINcVieuPH3fZ2e0P812?e=download
Resolving doc-10-1s-docs.googleusercontent.com (doc-10-1s-docs.googleusercontent.com)... 173.194.197.132, 2607:f8b0:4001:c1b::84
Connecting to doc-10-1s-docs.googleusercontent.com (d

In [4]:
!cp 79999_iter.pth face-parsing.PyTorch/res/cp 

In [5]:
#!/usr/bin/python
# -*- encoding: utf-8 -*-
import sys
sys.path.insert(1,'face-parsing.PyTorch')
from logger import setup_logger
from model import BiSeNet

import torch

import os
import os.path as osp
import numpy as np
import torchvision.transforms as transforms
import cv2

In [13]:

def evaluate(input_video,respth='./res/test_res', cp='model_final_diss.pth',kernel_size=(31,31),blur=True,background = None,fps=30):
  '''
  input_video : Path for the input video
  cp: Weight file
  kernel_size: Size of the Gaussian kernel for blurring
  blur: Set to True to perform blurring else the function performs background replacement
  background: Background to be used for replacement in the video, irrelevant if blur is set to True
  fps: Output frames per second
  '''

  if not os.path.exists(respth):
      os.makedirs(respth)

  n_classes = 19

  # loading network
  net = BiSeNet(n_classes=n_classes)
  net.cuda()
  save_pth = osp.join('face-parsing.PyTorch/res/cp', cp)
  net.load_state_dict(torch.load(save_pth))
  net.eval()

  # normailsation constants
  to_tensor = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
  ])

  # setting file names and reading background files
  if blur:
    output_file_name = 'blur_background_video.avi'
  else:
    output_file_name = 'change_background_video.avi'
    bg = cv2.imread(background)
    bg = cv2.resize(bg,(512,512), fx=1, fy=1)
    
  # video capture variables
  output_not_created = True
  vidcap = cv2.VideoCapture(input_video)
  iter = 1

  with torch.no_grad():
    # reading 
    has_frames, img = vidcap.read()
    while has_frames:

        # saving orginal size
        org_shape = (img.shape[1],img.shape[0])

        # resizing image for forward propagation
        img = cv2.resize(img,(512,512), fx=1, fy=1)
        img_ = to_tensor(img)
        img_ = torch.unsqueeze(img_, 0)
        img_ = img_.cuda()
        out = net(img_)[0]
        parsing = out.squeeze(0).cpu().numpy().argmax(0)

        if blur:
          bg = cv2.GaussianBlur(img,kernel_size,0)
        
        # resizing background and creating write object for video
        if output_not_created:
          output_not_created = False
          out_file = cv2.VideoWriter(output_file_name,cv2.VideoWriter_fourcc(*'DIVX'),fps,org_shape)
          

        # combining background and original image
        bg_flag = np.expand_dims(parsing,2) == 0
        new_img = bg*bg_flag + img*(~bg_flag)
        new_img = cv2.resize(new_img,org_shape)
        out_file.write(new_img)
        
        vidcap.set(cv2.CAP_PROP_POS_FRAMES,iter)
        iter += 1
        has_frames, img = vidcap.read()
        if iter %100 == 0:
          print(f'{iter} frames processed')

  return output_file_name

In [ ]:
evaluate(input_video='Taking Accountability_cut.mp4',cp='79999_iter.pth',blur=False,background='thumb-1920-126995.jpg')

100 frames processed
200 frames processed
300 frames processed
400 frames processed
500 frames processed
600 frames processed
700 frames processed
800 frames processed


In [11]:
evaluate(input_video='Taking Accountability_cut.mp4',cp='79999_iter.pth',blur=True)

100 frames processed
200 frames processed
300 frames processed
400 frames processed
500 frames processed
600 frames processed
700 frames processed
800 frames processed
900 frames processed
1000 frames processed
1100 frames processed
1200 frames processed
1300 frames processed
1400 frames processed
1500 frames processed
1600 frames processed
1700 frames processed
1800 frames processed
1900 frames processed
2000 frames processed
2100 frames processed
2200 frames processed


'blur_background_video.avi'